<a href="https://colab.research.google.com/github/Simarjit1303/Data-Science/blob/main/exercises/machine-learning/Neural-Network/multilayer_perceptrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilayer Perceptrons
You should build an end-to-end machine learning pipeline using a multilayer perceptron model. In particular, you should do the following:
- Load the `mnist` dataset using [Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). You can find this dataset in the datasets folder.
- Split the dataset into training and test sets using [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
- Build an end-to-end machine learning pipeline, including a [multilayer perceptron](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) model.
- Optimize your pipeline by validating your design decisions.
- Test the best pipeline on the test set and report various [evaluation metrics](https://scikit-learn.org/0.15/modules/model_evaluation.html).  
- Check the documentation to identify the most important hyperparameters, attributes, and methods of the model. Use them in practice.

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform, reciprocal
from sklearn.metrics import accuracy_score

In [16]:
df = pd.read_csv("https://raw.githubusercontent.com/m-mahdavi/teaching/refs/heads/main/datasets/mnist.csv")
df.head()

,id,class,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,31953,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,34452,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,60897,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,36953,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1981,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
print(df_train.shape, df_test.shape)

(3200, 786) (800, 786)


In [18]:
X_train = df_train.drop(['id'],axis=1)
y_train = df_train['class']
X_test = df_test.drop(['id'],axis=1)
y_test = df_test['class']

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(3200, 785) (3200,)
(800, 785) (800,)


## Feature Scaling

In [19]:
scaler =  StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape, X_test_scaled.shape)

(3200, 785) (800, 785)


### Model Training

### Default Parameter

In [20]:
multi_clf = MLPClassifier()
multi_clf.fit(X_train_scaled, y_train)
cv_score_default = cross_val_score(multi_clf, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Defult Mean_Cross_validation Score - {cv_score_default.mean()}")

y_pred_default = multi_clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_default)
print("Test Accuracy default:", accuracy)

Defult Mean_Cross_validation Score - 0.93
Test Accuracy default: 0.9225


### Hyperparameter tuning

In [21]:
param = {
    'hidden_layer_sizes' : [(50,50), (70,70), (100,)],
    'activation' : ['identity', 'logistic','relu', 'tanh'],
    'solver' : ['sgd', 'adam'],
    'alpha' : reciprocal(3e-5, 1e0),
    'batch_size' : ['auto', 300, 400],
    'learning_rate' : ['constant', 'adaptive'],
    'max_iter' : [1000, 2000, 3000],
    'tol' : [1e-4, 1e-5, 1e-6],
    'warm_start' : [True],
    'momentum' : [0.9, 0.95, 0.99, 1.0],
    'beta_1' : [0.9, 0.95, 0.99, 1.0],
    'beta_2' : [0.9, 0.95, 0.99, 1.0],
    'epsilon' : [1e-8, 1e-7, 1e-6, 1e-5],
    'random_state' : [42]
}

In [22]:
random_search_MLP = RandomizedSearchCV(MLPClassifier(), param, n_iter=10, cv=3, scoring = "accuracy", n_jobs=-1)
random_search_MLP.fit(X_train_scaled, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_p

RandomizedSearchCV(cv=3, estimator=MLPClassifier(), n_jobs=-1,
                   param_distributions={'activation': ['identity', 'logistic',
                                                       'relu', 'tanh'],
                                        'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fcdac504a90>,
                                        'batch_size': ['auto', 300, 400],
                                        'beta_1': [0.9, 0.95, 0.99, 1.0],
                                        'beta_2': [0.9, 0.95, 0.99, 1.0],
                                        'epsilon': [1e-08, 1e-07, 1e-06, 1e-05],
                                        'hidden_layer_sizes': [(50, 50),
                                                               (70, 70),
                                                               (100,)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'max_iter': [1000, 2000, 3000],
                                        'momentum': [0.9, 0.95, 0.99, 1.0],
                                        'random_state': [42],
                                        'solver': ['sgd', 'adam'],
                                        'tol': [0.0001, 1e-05, 1e-06],
                                        'warm_start': [True]},
                   scoring='accuracy')

In [23]:
best_params = random_search_MLP.best_params_
best_model = random_search_MLP.best_estimator_
best_score = random_search_MLP.best_score_

cv_score_mod = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='accuracy')


print("Best Parameters:", best_params)
print("Best Score:", best_score)
print(f"Mean -Cross_validation Score - {cv_score_mod.mean()}")

Best Parameters: {'activation': 'tanh', 'alpha': np.float64(0.930207874351402), 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.9, 'epsilon': 1e-08, 'hidden_layer_sizes': (70, 70), 'learning_rate': 'constant', 'max_iter': 1000, 'momentum': 0.95, 'random_state': 42, 'solver': 'adam', 'tol': 1e-05, 'warm_start': True}
Best Score: 0.9437511612517898
Mean -Cross_validation Score - 0.9556250000000001


In [24]:
y_pred = best_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.95125
